In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import os

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [25]:
COLUMN_OF_INTEREST =['QUOTE_UNIXTIME','QUOTE_DATE','EXPIRE_DATE','EXPIRE_UNIX',
                            'DTE','C_BID','C_ASK', 'P_BID','P_ASK',
                             'UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE']

In [26]:
# def read_data(path):
#     df = pd.read_csv(path)

#     df.columns = df.columns.str.replace(' ', '')
#     df.columns = df.columns.str.replace('[', '', regex=True)
#     df.columns = df.columns.str.replace(']', '', regex=True)

#     df_of_interest = df[COLUMN_OF_INTEREST]
#     df_of_interest = df_of_interest[(df_of_interest['DTE'] == 30) | (df_of_interest['DTE'] == 0)]

#     return df_of_interest

In [27]:
# # create empty dataframe with column names
# df_of_interest = pd.DataFrame(columns=COLUMN_OF_INTEREST)

# #loop through all files in directory
# for filename in tqdm(os.listdir('data/2020 to 2022 txt files')):
#     if not filename.endswith(".txt"):
#         continue
#     path = os.path.join('./data/2020 to 2022 txt files', filename)
#     df_of_interest = pd.concat([df_of_interest, read_data(path)])


In [28]:
# write to csv
# df_of_interest.to_csv('data/2020-2022_30days.csv', index=False)

In [29]:
df_options = pd.read_csv('data/2020-2022_30days.csv')

# convert QUOTE_DATE and EXPIRE_DATE to datetime
df_options['QUOTE_DATE'] = pd.to_datetime(df_options['QUOTE_DATE'])
df_options['EXPIRE_DATE'] = pd.to_datetime(df_options['EXPIRE_DATE'])

# drop QUOTE_UNIXTIME and EXPIRE_UNIX
df_options = df_options.drop(['QUOTE_UNIXTIME', 'EXPIRE_UNIX'], axis=1)

# set "" to NaN
df_options = df_options.replace(r'^\s*$', np.nan, regex=True)

# drop rows with NaN
df_options = df_options.dropna()

# convert C_BID, C_ASK, P_BID, P_ASK to float
df_options['C_BID'] = df_options['C_BID'].astype(float)
df_options['C_ASK'] = df_options['C_ASK'].astype(float)
df_options['P_BID'] = df_options['P_BID'].astype(float)
df_options['P_ASK'] = df_options['P_ASK'].astype(float)

In [30]:
df_options.dtypes

QUOTE_DATE         datetime64[ns]
EXPIRE_DATE        datetime64[ns]
DTE                       float64
C_BID                     float64
C_ASK                     float64
P_BID                     float64
P_ASK                     float64
UNDERLYING_LAST           float64
STRIKE                    float64
STRIKE_DISTANCE           float64
dtype: object

In [31]:
df_options.head()

,QUOTE_DATE,EXPIRE_DATE,DTE,C_BID,C_ASK,P_BID,P_ASK,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE
0,2020-01-03,2020-01-03,0.0,1826.61,1849.20,0.0,0.05,3234.35,1400.0,1834.3
1,2020-01-03,2020-01-03,0.0,1726.60,1749.30,0.0,0.05,3234.35,1500.0,1734.3
2,2020-01-03,2020-01-03,0.0,1626.60,1649.19,0.0,0.04,3234.35,1600.0,1634.3
3,2020-01-03,2020-01-03,0.0,1526.60,1549.31,0.0,0.05,3234.35,1700.0,1534.3
4,2020-01-03,2020-01-03,0.0,1426.60,1449.30,0.0,0.04,3234.35,1800.0,1434.3


In [155]:
# Calculate the rolling volatility of the underlying price
import yfinance as yf
from pypfopt.risk_models import CovarianceShrinkage

TICKER = ['SPY']

start_date = '2017-01-01' # you need to have at least 2 years of data before the start date as we are calculating the rolling volatility of 2 years
end_date = '2023-01-01' # end date is exclusive, the data will be downloaded until 2022-12-31

df = yf.download(TICKER, start=start_date, end=end_date)

# calculate the rolling volatility of 2 years (252 trading days per year)
rolling_period = 252*2
dt = 1/252

# VOLATILITY METHOD 1: calculate the volatility using log return and take the standard deviation as is
# df['log_return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
# df['volatility'] = df['log_return'].rolling(rolling_period).std() * np.sqrt(252)

# VOLATILITY METHOD 2: calculate the volatility using log return and CovarianceShrinkage
df_spy = df['Adj Close']
df_spy.index.name = None
df['volatility'] = np.nan

#Apply rolling period of 2 years to CovarianceShrinkage
for i in range(rolling_period, len(df_spy)):
    cov_matrix = CovarianceShrinkage(df_spy.iloc[i-rolling_period:i], log_returns=True).ledoit_wolf() 

    #calculate the volatility. cov_matrix is annualized, so no need to multiply by sqrt(252)
    df['volatility'].iloc[i] = np.sqrt(cov_matrix.iloc[0,0])
    
# we only need the data from 2020 to 2022
df = df.loc['2020-01-01':'2022-12-31']

[*********************100%***********************]  1 of 1 completed


In [156]:
# risk free rate for 2020 to 2022 is around 2.5% per annum
risk_free_rate = 0.025

# dividend yield is rate is roughly 1.5% per annum for SPY
dividend_yield = 0.015

# time interval for binomial model is 1 day, so we multiply the volatility by sqrt(1/252)
# calculate factor change of upstate (Cox-Ross-Rubinstein model)
df['u_crr'] = np.exp(df['volatility'] * np.sqrt(dt))

# calculate factor change of upstate (Jarrow-Rudd model)
df['u_jr'] = np.exp((risk_free_rate - 0.5 * df['volatility']**2) * dt + df['volatility'] * np.sqrt(dt))

# calculate factor change of upstate (Jarrow-Rudd model) accounting for dividend yield
df['u_jr_div'] = np.exp((risk_free_rate - dividend_yield - 0.5 * df['volatility']**2) * dt + df['volatility'] * np.sqrt(dt))



In [157]:
# Binomial model 
def binomial_model(N, S0, u, r, K, dt, div_yield=0, option_type='call', model='crr', european=True):
    """
    N: number of steps
    S0: initial stock price
    u: factor change of upstate
    r: risk free rate
    div_yield: dividend yield
    K: strike price
    option_type: 'call' or 'put'
    model: 'crr', 'jr'
    """

    # calculate factor change of downstate
    d = 1/u

    # calculate probability of upstate
    if model == 'crr':
        p = (np.exp((r - div_yield) * dt) - d) / (u - d)
    elif model == 'jr':
        p = 0.5

    # calculate probability of downstate
    q = 1 - p

    # calculate stock price at each node
    stock_price = np.zeros((N+1, N+1))
    stock_price[0, 0] = S0
    for i in range(1, N+1):
        for j in range(i+1):
            stock_price[j, i] = S0 * u**(i-j) * d**j
  
    # calculate option price at final nodes
    option_price = np.zeros((N+1, N+1))
    if option_type == 'call':
        option_price[:, N] = np.maximum(stock_price[:, N] - K, 0)
    elif option_type == 'put':
        option_price[:, N] = np.maximum(K - stock_price[:, N], 0)

    # calculate option price at each node
    if european:
        for i in reversed(range(N)):
            for j in range(i+1):
                option_price[j, i] = np.exp(-r * dt) * (p * option_price[j, i+1] + q * option_price[j+1, i+1])
    else:
        if option_type == 'call':
            for i in reversed(range(N)):
                for j in range(i+1):
                    option_price[j, i] = np.maximum(np.exp(-r * dt) * (p * option_price[j, i+1] + q * option_price[j+1, i+1]), stock_price[j, i] - K)
        elif option_type == 'put':
            for i in reversed(range(N)):
                for j in range(i+1):
                    option_price[j, i] = np.maximum(np.exp(-r * dt) * (p * option_price[j, i+1] + q * option_price[j+1, i+1]), K - stock_price[j, i])
    return option_price[0, 0]

In [158]:
print('European Option')
price = binomial_model(30, 20, 0.5, 10.25*252, 8, 1/252, option_type='call', model='crr', european=True)
print(price)

print('American Option')
price = binomial_model(30, 20, 0.5, 10.25*252, 8, 1/252, option_type='call', model='crr', european=False)
print(price)


European Option
123.02212768340627
American Option
34.41578303436046


In [159]:
# merge the dataframes of df_options and df where df_options QUOTE_DATE is same as df index
df_model_input = df_options.merge(df, left_on='QUOTE_DATE', right_on=df.index)
df_model_input = df_model_input[df_model_input['DTE']==30]

In [160]:
df_model_input.head()

,QUOTE_DATE,EXPIRE_DATE,DTE,C_BID,C_ASK,P_BID,P_ASK,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,...,High,Low,Close,Adj Close,Volume,log_return,volatility,u_crr,u_jr,u_jr_div
382,2020-01-06,2020-02-05,30.0,1241.90,1248.39,0.00,0.14,3246.23,2000.0,1246.2,...,323.730011,320.359985,323.640015,308.522339,55653900,0.003807,0.149907,1.009488,1.009543,1.009483
383,2020-01-06,2020-02-05,30.0,1142.89,1148.59,0.10,0.16,3246.23,2100.0,1146.2,...,323.730011,320.359985,323.640015,308.522339,55653900,0.003807,0.149907,1.009488,1.009543,1.009483
384,2020-01-06,2020-02-05,30.0,1092.10,1098.70,0.10,0.20,3246.23,2150.0,1096.2,...,323.730011,320.359985,323.640015,308.522339,55653900,0.003807,0.149907,1.009488,1.009543,1.009483
385,2020-01-06,2020-02-05,30.0,1043.10,1048.80,0.05,0.20,3246.23,2200.0,1046.2,...,323.730011,320.359985,323.640015,308.522339,55653900,0.003807,0.149907,1.009488,1.009543,1.009483
386,2020-01-06,2020-02-05,30.0,993.20,998.91,0.10,0.21,3246.23,2250.0,996.2,...,323.730011,320.359985,323.640015,308.522339,55653900,0.003807,0.149907,1.009488,1.009543,1.009483


In [161]:
# backtest the binomial model
def backtest_binomial_model(df, N, risk_free_rate, dt, dividend_yield=0, option_type='call', model='crr', european=True):
    """
    df: dataframe of options data
    N: number of steps
    risk_free_rate: risk free rate
    dt: time interval
    dividend_yield: dividend yield
    option_type: 'call' or 'put'
    model: 'crr', 'jr'
    european: True or False (True for European option, False for American option)
    """
    df['binomial_model'] = 0
    if model == 'crr':
        for i in tqdm(range(len(df))):
            df['binomial_model'].iloc[i] = binomial_model(N, df['UNDERLYING_LAST'].iloc[i], df['u_crr'].iloc[i], risk_free_rate, df['STRIKE'].iloc[i], dt, div_yield=dividend_yield, option_type=option_type, model=model, european=european)
    elif model == 'jr':
        for i in tqdm(range(len(df))):
            df['binomial_model'].iloc[i] = binomial_model(N, df['UNDERLYING_LAST'].iloc[i], df['u_jr'].iloc[i], risk_free_rate, df['STRIKE'].iloc[i], dt, div_yield=dividend_yield, option_type=option_type, model=model, european=european)
    return df

In [162]:

df_model_output_crr = backtest_binomial_model(df_model_input.copy(), 
                                          21, 
                                          risk_free_rate, dt, 
                                          dividend_yield, 
                                          option_type='call', model='crr', 
                                          european=True)

  0%|          | 0/38928 [00:00<?, ?it/s]

In [163]:
df_model_output_crr[['binomial_model','C_BID','C_ASK']].head(8)

,binomial_model,C_BID,C_ASK
382,1246.337077,1241.90,1248.39
383,1146.545193,1142.89,1148.59
384,1096.649252,1092.10,1098.70
385,1046.753310,1043.10,1048.80
386,996.857368,993.20,998.91
387,946.961426,942.50,949.00
388,897.065485,893.60,900.01
389,847.169543,842.70,849.29


In [164]:

df_model_output_jr = backtest_binomial_model(df_model_input.copy(), 
                                          21, 
                                          risk_free_rate, dt, 
                                          dividend_yield, 
                                          option_type='call', model='jr', 
                                          european=True)

  0%|          | 0/38928 [00:00<?, ?it/s]

In [165]:
df_model_output_jr[['binomial_model','C_BID','C_ASK']].head(8)

,binomial_model,C_BID,C_ASK
382,1246.706225,1241.90,1248.39
383,1146.914341,1142.89,1148.59
384,1097.018400,1092.10,1098.70
385,1047.122458,1043.10,1048.80
386,997.226516,993.20,998.91
387,947.330574,942.50,949.00
388,897.434633,893.60,900.01
389,847.538691,842.70,849.29


In [166]:
# merge df_model_output_crr and df_model_output_jr on index 
df_model_output = df_model_output_crr.merge(df_model_output_jr, left_index=True, right_index=True, suffixes=('_crr', '_jr'))


In [167]:
temp = df_model_output[['binomial_model_crr','binomial_model_jr','C_BID_jr','C_ASK_jr']]
temp.head()

,binomial_model_crr,binomial_model_jr,C_BID_jr,C_ASK_jr
382,1246.337077,1246.706225,1241.90,1248.39
383,1146.545193,1146.914341,1142.89,1148.59
384,1096.649252,1097.018400,1092.10,1098.70
385,1046.753310,1047.122458,1043.10,1048.80
386,996.857368,997.226516,993.20,998.91


In [168]:
#convert to float
temp.dropna(inplace=True)
temp['binomial_model_crr'] = temp['binomial_model_crr'].astype(float)
temp['binomial_model_jr'] = temp['binomial_model_jr'].astype(float)
temp['C_BID_jr'] = temp['C_BID_jr'].astype(float)
temp['C_ASK_jr'] = temp['C_ASK_jr'].astype(float)

temp['C_PRC'] = (temp['C_BID_jr'] + temp['C_ASK_jr'])/2
temp.head()

,binomial_model_crr,binomial_model_jr,C_BID_jr,C_ASK_jr,C_PRC
382,1246.337077,1246.706225,1241.90,1248.39,1245.145
383,1146.545193,1146.914341,1142.89,1148.59,1145.740
384,1096.649252,1097.018400,1092.10,1098.70,1095.400
385,1046.753310,1047.122458,1043.10,1048.80,1045.950
386,996.857368,997.226516,993.20,998.91,996.055


In [169]:
# calculate error
temp['error_crr'] = temp['binomial_model_crr'] - temp['C_PRC']
temp['error_jr'] = temp['binomial_model_jr'] - temp['C_PRC']

# print root mean squared error
print('Root Mean Squared Error of CRR model: ', np.sqrt(np.mean(temp['error_crr']**2)))
print('Root Mean Squared Error of JR model: ', np.sqrt(np.mean(temp['error_jr']**2)))

# print mean absolute error
print('Mean Absolute Error of CRR model: ', np.mean(np.abs(temp['error_crr'])))
print('Mean Absolute Error of JR model: ', np.mean(np.abs(temp['error_jr'])))

# print percentage of error
print('Percentage of Error of CRR model: ', np.mean(np.abs(temp['error_crr']))/np.mean(temp['C_PRC'])*100)
print('Percentage of Error of JR model: ', np.mean(np.abs(temp['error_jr']))/np.mean(temp['C_PRC'])*100)

Root Mean Squared Error of CRR model:  23.86935498966928
Root Mean Squared Error of JR model:  25.103188203551444
Mean Absolute Error of CRR model:  14.540254685604587
Mean Absolute Error of JR model:  16.29314434180136
Percentage of Error of CRR model:  3.081296435831501
Percentage of Error of JR model:  3.4527598501135346


In [170]:
# METHOD 1: using std as is
# Root Mean Squared Error of CRR model:  23.86935498966928
# Root Mean Squared Error of JR model:  25.103188203551444
# Mean Absolute Error of CRR model:  14.540254685604587
# Mean Absolute Error of JR model:  16.29314434180136
# Percentage of Error of CRR model:  3.081296435831501
# Percentage of Error of JR model:  3.4527598501135346


# METHOD 2: using COVARIANCE_SHRINKAGE
# Root Mean Squared Error of CRR model:  23.920759846458463
# Root Mean Squared Error of JR model:  25.15368471620861
# Mean Absolute Error of CRR model:  14.557521681224893
# Mean Absolute Error of JR model:  16.30927181380145
# Percentage of Error of CRR model:  3.0849555692657344
# Percentage of Error of JR model:  3.456177501527993